In [1]:
# Install Required Libraries

!pip install pillow


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import Libraries


from PIL import Image
import numpy as np

In [3]:
# ASCII XOR + LSB Steganography Functions


def xor_encrypt(message, key):
    return ''.join([chr(ord(c) ^ ord(key)) for c in message])

def xor_decrypt(cipher_text, key):
    return ''.join([chr(ord(c) ^ ord(key)) for c in cipher_text])

def text_to_binary(text):
    return ''.join(format(ord(char), '08b') for char in text)

def binary_to_text(binary):
    chars = [binary[i:i+8] for i in range(0, len(binary), 8)]
    return ''.join([chr(int(char, 2)) for char in chars])

In [7]:
# Encode Message into Image using LSB


def encode_lsb(image_path, message, key, output_path):
    img = Image.open(image_path).convert("RGB")  # force RGB mode
    encoded = img.copy()
    width, height = img.size

    message = xor_encrypt(message, key)
    binary_msg = text_to_binary(message) + '1111111111111110'  # End delimiter

    data_index = 0
    for y in range(height):
        for x in range(width):
            if data_index < len(binary_msg):
                pixel = list(encoded.getpixel((x, y)))
                for n in range(3):  # R, G, B
                    if data_index < len(binary_msg):
                        pixel[n] = pixel[n] & ~1 | int(binary_msg[data_index])
                        data_index += 1
                encoded.putpixel((x, y), tuple(pixel))

    encoded.save(output_path)
    print("✅ Message encoded and saved as", output_path)


In [8]:
# Decode Hidden Message from Image


def decode_lsb(stego_image_path, key):
    img = Image.open(stego_image_path).convert("RGB")
    binary_data = ""
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            pixel = img.getpixel((x, y))
            for n in range(3):
                binary_data += str(pixel[n] & 1)

    all_bytes = [binary_data[i:i+8] for i in range(0, len(binary_data), 8)]
    decoded_chars = []
    for byte in all_bytes:
        if byte == '11111110':  # End delimiter
            break
        decoded_chars.append(chr(int(byte, 2)))

    decrypted = xor_decrypt(''.join(decoded_chars), key)
    return decrypted


In [9]:
# Testing


message = "Hello Jasmin, your internship is on point!"
key = "K"

encode_lsb("image.png", message, key, "stego_image.png")
decoded = decode_lsb("stego_image.png", key)
print("🔓 Decoded Message:", decoded)



✅ Message encoded and saved as stego_image.png
🔓 Decoded Message: Hello Jasmin, your internship is on point!´
